In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from scipy.stats import norm
from scipy import optimize

# Some Basics: Caps and Floors

## Cap

A <i>Cap</i> is made up with a trip of <i>caplets</i>. A caplet is a derivatives on interest rate, much like a call option on interest rate such that it protects the buyer from rising interest rate. For a caplet with reset date at $T_0$ and a settlement date at $T_1$, the cashflow at $T_1$ for the caplet holder is as such:

$$CF( T_1) = \delta (L(T_0, T_1) - \kappa)^+$$<br>

The time-$T_0$ value of the caplet is then 
$$V(T_0, T_1) = P(T_0, T_1)\delta (L(T_0,T_1)- \kappa)^+$$<br>

Rearranging, we get 
$$V(T_0, T_1) = (1+\delta \kappa)(\frac{1}{1+\delta \kappa} - P(T_0, T_1))^+ = (1+\delta \kappa) \times CF_{put}$$, 

where $CF_{put}$ is the cashflow of a put option on a $T_1$-Bond with expiry $T_0$ and a strike of $\frac{1}{1+\delta \kappa}$.<br>

Thus, the Time-t price of the caplet is given by 
$$Cpl(t,T_0,T_1) = (1+\delta \kappa)\times p_{put}$$<br>

A Cap is specified by:
- reset/ settlement dates $T_0 < ... < T_N$
- a cap rate $\kappa$

Assuming a cap with caplets which settlement dates are of equal interval ($\delta$) away from the previous caplet, the cap price at time $t < T_0$ is given by:<br>
$$Cp(t) = \sum_{i=1}^n Cpl(t, T_{i-1}, T_i)$$<br>

## Floor

Conversely, a floor is made up of a strip of floorlets and it seeks to protect holder from falling interest rates.
Cashflow of floorlets at time $T_1$ is $\delta (\kappa - L(T_{i-1}, T_{i}))^+$<br>
$$Fl(t) = \sum_{i=1}^n Fll(t, T_{i-1}, T_i)$$<br>

## Cap-Floor Parity

The following Cap-Floor Parity holds:
$$Cp(t) - Fl(t) = V_p(t)$$

Where $V_p(t)$ is the time-t value of a payer swap with fixed rate $\kappa$, a notional value of 1, and the same tenor as the cap and floor.<br>

The cap(floor) is said to be
- <b>At-the-money (ATM)</b> if $\kappa = R_{swap}(t) = \frac{P(t,T_0) - P(t,T_n)}{\delta \sum_{i=1}^n P(t,T_i)}$
- <b>In-the-money (ITM)</b> if cap: $\kappa < R_{swap}(t)$, floor: $\kappa > R_{swap}(t)$
- <b>Out-of-the-money (OTM)</b> if cap: $\kappa > R_{swap}(t)$, floor: $\kappa < R_{swap}(t)$

---

# Exercise: Cap Pricing using Black's and Bachelier's Formulae

Example derived from 'Interest Rate Models' course provided by EPFL on Coursera.<br>

Problem Statement:<br>
Suppose at $t = 0$,we observed the following forward rates<br>

| T_0 | T_i | F |
| --- | --- | --- |
| 0 | $\frac{1}{4}$ | 6\% |
| $\frac{1}{4}$ | $\frac{1}{2}$ | 8\% |
| $\frac{1}{2}$ | $\frac{3}{4}$ | 9\% |
| $\frac{3}{4}$ | 1 | 10\% |
| 1 | $\frac{5}{4}$ | 10\% |
| $\frac{5}{4}$ | $\frac{3}{2}$ | 10\% |
| $\frac{3}{2}$ | $\frac{7}{4}$ | 9\% |
| $\frac{7}{4}$ | 2 | 9\% |

<br>

Consider at-the-money cap with reset dates $\frac{i}{4}$, where $i = 1,2,...,8$, solve:<br>
1. For the implied Black volatility, suppose that the price of cap is 1\%
2. Suppose the same cap price, find the implied normal volatility (in bps)
3. Obtain the cap price, assuming that the implied Black volatility is 14.1\%

In [2]:
# Set up given information into variables
delta = 0.25
forward_rates = [0.06, 0.08, 0.09, 0.10, 0.10, 0.10, 0.09, 0.09]

In [3]:
# Solve for discount curve (required for swap rate calculation)
discount_prices = np.zeros(1+ len(forward_rates))
discount_prices[0] = 1

for i in range(8):
    discount_prices[i+1] = discount_prices[i]/(1+delta*forward_rates[i])
print(discount_prices)

[1.         0.98522167 0.9659036  0.944649   0.92160878 0.89913052
 0.87720051 0.8578978  0.83901986]


In [4]:
# Calculate swap rates for the different tenors
R_swap = np.zeros(7)

for i in range(len(R_swap)):
    R_swap[i] = (discount_prices[1] - discount_prices[i+2])/(delta*np.sum(discount_prices[2:i+3]))

R_swap

array([0.08      , 0.08494438, 0.0898436 , 0.09229099, 0.09375836,
       0.09316852, 0.0927469 ])

# Cap Price by Black's Formula


Recall that a cap is a strip of caplets, which is specified by:
- reset/ settlement dates $T_0 < ... < T_n$
- a cap rate $\kappa$
- assume fixed interval $\delta = T_i - T_{i-1}$

$$Cp(t) = \sum_{i=1}^n Cpl(t, T_{i-1}, T_i)$$

Black's Formula has the following assumptions:<br>
Simple Spot Rate $L(T_{i-1}, T_i) = F(T_{i-1}, T_{i-1},T_i)$ is log-normal, with constant $\sigma > 0$ and Brownian motion $W^{T_i}(t)$ under the $T_i$-forward measure.

Where, by Black's Formula, <br>
$$Cpl(t, T_{i-1}, T_i) = \delta P(t,T_i)(F(t, T_{i-1},T_i)\Phi(d_1) - \kappa \Phi(d_2)), i \ge 1$$

$$d_{1,2} = \frac{log(\frac{F(t,T_{i-1}, T_i)}{\kappa}) \pm 0.5 \sigma^2(T_{i-1}-t)}{\sigma \sqrt{T_{i-1}-t}}$$
$\Phi(\cdot)$ is the cumulative distribution function (cdf) of $(\cdot)$

In [5]:
def get_cap_price(discount_prices, forward_rates, strike, black_vola, periods, delta):
    caplets = get_caplet_prices(discount_prices, forward_rates, strike, black_vola, periods, delta)
    return np.sum(caplets)

In [6]:
def get_caplet_prices(discount_prices, forward_rates, strike, black_vola, periods, delta):
    d1, d2 = get_d_params(forward_rates, strike, black_vola, periods)
    
    return delta*discount_prices*(forward_rates*norm.cdf(d1)-strike*norm.cdf(d2))

In [7]:
def get_d_params(forward_rates, strike, black_vola, periods):
    # forward_rates should begin from first reset date
    # periods should begin from first reset date, end one step before last period
    numerator_1 = np.log(forward_rates/strike)
    numerator_2 = 0.5*black_vola**2*periods
    denominator = black_vola*np.sqrt(periods)
    
    d1 = (numerator_1 + numerator_2)/denominator
    d2 = (numerator_1 - numerator_2)/denominator
    
    return d1, d2

Given that the cap is priced at 1\%, we will solve the root-solving optimization problem using Brent's method. We set the range between 0 and 1, as we expect the implied Black volatility to fall within this range.

In [8]:
def solve_black_vola(targets, Tn, discount_prices_s, forward_rates_s, periods, delta, strikes):
    implied_volatility = []
    for idx, n in enumerate(Tn):
        print('====================== Initiating solution for case %s ======================' % (idx+1))
        i = int(n/delta - 1)
        discount_prices_required = discount_prices_s[0:i]
        forward_rates_required = forward_rates_s[0:i]
        periods_required = periods[0:i]
        strike = strikes[i-1]
        
        sol = optimize.root_scalar(lambda vol: get_cap_price(discount_prices_required, forward_rates_required,\
                                                 strike, vol, periods_required, delta) - targets[idx],\
                                   bracket=[0, 1], method='brentq')
        print('Cap market price = {0}. Solution for implied volatility (black) = {1}'.format(targets[idx], sol.root))
        implied_volatility.append(sol.root)
    return implied_volatility

In [9]:
targets =[0.01]
Tn = [2]
periods = np.arange(delta, 9*delta, delta)

discount_prices_s = discount_prices[2:]
forward_rates_s = forward_rates[1:]
strikes = R_swap

black_volatilities = solve_black_vola(targets, Tn, discount_prices_s, forward_rates_s, periods, delta, strikes)

====================== Initiating solution for case 1 ======================
Cap market price = 0.01. Solution for implied volatility (black) = 0.15347230483049518


From our solution, the implied Black volatlity is 15.35\% for a cap price of 1\%.

In [10]:
# Solve for cap price given implied Black volatility of 14.1%
periods_s = periods[0:7] 
discount_prices_s = discount_prices[2:9]
forward_rates_s = forward_rates[1:8]
strike = R_swap[-1]

cap_price = get_cap_price(discount_prices_s, forward_rates_s, strike, 0.141, periods_s, delta)
print('The price of cap with implied Black volatility of 14.1% is {}%.'.format(round(cap_price*100,2)))

The price of cap with implied Black volatility of 14.1% is 0.94%.


---

# Cap Price by Bachelier's Formula

Bachelier's formula assumes that $L(T_{i-1},T_i) = F(T_{i-1}, T_{i-1}, T_i)$ is normal, with $dF(t, T_{i-1}, T_i) = \sigma dW^{T_i}(t)$, with constant $\sigma > 0$ and Brownian motion $W^{T_i}(t)$ under the $T_i$-forward measure.<br>

$$Cp(t) = \sum_{i=1}^n Cpl(t, T_{i-1}, T_i)$$

Where, by Bachelier's Formula, <br>
$$Cpl(t, T_{i-1}, T_i) = \delta P(t,T_i)\sigma \sqrt{T_{i-1}-t}(D\Phi (D) + \phi(D)), i \ge 1$$
And
$$D = \frac{F(t,T_{i-1},T_i) - \kappa}{\sigma \sqrt{T_{i-1}-t}}$$

In [11]:
def get_bach_cap_price(discount_prices, forward_rates, strike, bach_vola, periods, delta):
    caplets = get_bachelier_caplet_prices(discount_prices, forward_rates, strike, bach_vola, periods, delta)

    return np.sum(caplets)

In [12]:
def get_bachelier_caplet_prices(discount_prices, forward_rates, strike, bach_vola, periods, delta):
    D = get_D_params(forward_rates, strike, bach_vola, periods)

    return delta*discount_prices*bach_vola*np.sqrt(periods)*(D*norm.cdf(D)+norm.pdf(D))

In [13]:
def get_D_params(forward_rates, strike, bach_vola, periods):

    numerator = (forward_rates - strike).reshape(-1,1)
    denominator = (bach_vola*np.sqrt(periods)).reshape(-1,1)
    
    return numerator / denominator

In [14]:
def solve_bach_vola(targets, Tn, discount_prices_s, forward_rates_s, periods, delta, strikes):
    implied_volatility = []
    for idx, n in enumerate(Tn):
        print('====================== Initiating solution for case %s ======================' % (idx+1))
        i = int(n/delta - 1)
        discount_prices_required = np.array(discount_prices_s[0:i]).reshape(-1,1)
        forward_rates_required = np.array(forward_rates_s[0:i]).reshape(-1,1)
        periods_required = np.array(periods[0:i]).reshape(-1,1)
        strike = np.array(strikes[i-1]).reshape(-1,1)
        
        sol = optimize.root_scalar(lambda vol: get_bach_cap_price(discount_prices_required, forward_rates_required,\
                                                 strike, vol, periods_required, delta) - targets[idx],\
                                   bracket=[0.0001, 0.5], method='brentq')
        print('Cap market price = {0}. Solution for implied volatility (bach) = {1}'.format(targets[idx], sol.root))
        implied_volatility.append(sol.root)
    return implied_volatility

In [15]:
targets =[0.01]
Tn = [2]
periods = np.arange(delta, 9*delta, delta)

discount_prices_s = discount_prices[2:]
forward_rates_s = forward_rates[1:]
strikes = R_swap

bach_volatilities = solve_bach_vola(targets, Tn, discount_prices_s, forward_rates_s, periods, delta, strikes)

print('\nThe implied normal volatility of a cap with price of 1.00% is {} bps'.format(round(bach_volatilities[0]*100**2,2)))

====================== Initiating solution for case 1 ======================
Cap market price = 0.01. Solution for implied volatility (bach) = 0.014335991583284793

The implied normal volatility of a cap with price of 1.00% is 143.36 bps
